# Quickstart

> This page provides a fast introduction to FastHTML that covers most of what you need to be useful. This document is a work in progress, 

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

## Installation

In [ ]:
pip install python-fasthtml

## A Minimal Application

A minimal FastHTML application looks something like this:

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@app.get("/")
def home():
    return Titled("FastHTML",
        P("Let's do this!"))

run_uv()
```

What does that code do?

1. First we imported the FastHTML namespace on line 1. This is a carefully specified set of functions and other Python objects designed to optimize development of FastHTML projects. 

2. Next on line 3 we instantiate the FastHTML app with the `fast_app()` utility function. This provides a number of really useful defaults that we'll take advantage of later in the tutorial. 

3. Then on line 5 we use the `app.get()` decorator to tell FastTML what URL should trigger our view function if visited with an HTTP GET. By HTTP GET we mean "go to that location in our browser"

4. Line 6 is where we define our function name.

5. On line 7 we return several functions that describe all the HTML required to write a properly formed web page.

6. Finally, on line 10 the `run_uv()` utility configures and runs FastHTML using a library called `uvicorn`.

Run the code:

```bash
python main.py
```

The terminal will look like this:

```bash
INFO:     Uvicorn running on http://0.0.0.0:5001 (Press CTRL+C to quit)
INFO:     Started reloader process [58058] using WatchFiles
INFO:     Started server process [58060]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
```

Confirm FastHTML is running by opening your web browser to this linl: [127.0.0.1:5001](http://127.0.0.1:5001). You should see something like the image below:

![](quickstart-fasthtml.png)

We did it! Now that we've got the program running, let's view the source of the HTML page. It should look something like this:

```html
<!doctype html>

<html>
  <head>
    <title>FastHTML</title>
    <meta charset="utf-8"></meta>
    <meta name="viewport" content="width=device-width, initial-scale=1, viewport-fit=cover"></meta>
    <script src="https://unpkg.com/htmx.org@next/dist/htmx.min.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/answerdotai/surreal@1.3.0/surreal.js"></script>
    <script src="https://cdn.jsdelivr.net/gh/gnat/css-scope-inline@main/script.js"></script>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/@picocss/pico@latest/css/pico.min.css">
    <style>:root { --pico-font-size: 100%; }</style>
  </head>
  <body>
<main class="container">
  <h1>FastHTML</h1>
  <p>Let&#x27;s do this!</p>
</main>
  </body>
</html>

```

## A Minimal Charting Application

```python
import json
from fasthtml.common import * 

app, rt = fast_app(hdrs=(Script(src="https://cdn.plot.ly/plotly-2.32.0.min.js"),))

data = json.dumps({
    "data": [
            {
            "x": [1, 2, 3, 4],
            "type": "scatter"
            },

            {
            "x": [1, 2, 3, 4],
            "y": [16, 5, 11, 9],
            "type": "scatter"
            }
        ],
    "title": "Plotly chart in FastHTML ",
    "description": "This is a demo dashboard",
    "type": "scatter"
})


@app.get("/")
def home():
  return Titled("FastHTML", P("Let's do this!"), Div(id="myDiv"),
    Script(f"var data = {json.loads(data)}; Plotly.newPlot('myDiv', data);"))

run_uv()
```

## Routing

Friendly URLs are intuitive and helpful to users, and easier to maintain for coders. Users are more likely to use your project if they can remember URLs without much effort. Fortunately, FastHTML uses the Python community's common pattern for specifying URLs.

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@app.get("/")  # <1>
def home():
  return Titled("FastHTML", P("Let's do this!"))

@app.get("/hello")  # <2>
def hello():
  return Titled("Hello, world!")

run_uv()
```

1. The "/" URL on line 5 is the home of a project. This would be accessed at [127.0.0.1:5001](http://127.0.0.1:5001).
2. "/hello" URL on line 9 will be found by the project if the user visits [127.0.0.1:5001/hello](http://127.0.0.1:5001/hello).

You can do more! Read on to learn what we can do to make parts of the URL dynamic.

## Variable in URLs

You can add variable sections to a URL by marking sections with `{variable_name}`. Your function then receives the `{variable_name}` as a keyword argument, but only if it is the correct type. Here's an example:

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@app.get("/")
def home():
  return Titled("FastHTML", P("Let's do this!"))

@app.get("/{name}/{age}")  # <3>
def namer(name: str, age: int):  # <4>
  return Titled(f"Hello {name.title()}, age {age}")  # <5>

run_uv()
```

3. On line 9 we specify two variable names, `name` and `age`
4. On line 10 we define two function arguments named identically to the variables. You will note that we specify the Python types to be passed
5. On line 11, we use these functions in our project

Try it out by going to this address: [127.0.0.1:5001/uma/5](http://127.0.0.1:5001/uma/5). You should get a page that says,

> "Hello Uma, age 5".


### What happens if we enter incorrect data?

The [127.0.0.1:5001/uma/5](http://127.0.0.1:5001/uma/5) URL works because `5` is an integer. If we enter something that is not, such as [127.0.0.1:5001/uma/five](http://127.0.0.1:5001/uma/five), then FastHTML will return an error instead of a web page.

::: {.callout-note}
### FastHTML URL routing supports more complex types

The two examples we provide here use Python's built-in `str` and `int` types, but you can use your own types, including more complex ones such as those defined by libraries like [attrs](https://pypi.org/project/attrs/), [pydantic](https://pypi.org/project/pydantic/), and even [sqlmodel](https://pypi.org/project/attrs/). 
:::


## HTTP Methods

Most commonly URL routes for web apps are defined as HTTP GET methods. However, form submissions often are sent as HTTP POST. When dealing with more dynamic web page designs, also known as Single Page Apps (SPA for short), the need can arise for other methods such as HTTP PUT and HTTP DELETE. The way FastHTML handles this is by changing the decorator.

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app()

@app.get("/")  # <6>
def handle_get():
  return Titled("HTTP GET", P("Handle GET"))

@app.post("/")  # <7>
def handle_post():
  return Titled("HTTP POST", P("Handle POST"))

run_uv()
```

6. On line 5 the `app.get()` decorator is used
7. On line 10 the `app.post()` decorator is used






## Static Files

FastHTML automatically handles common usecase for static media. That is to say, reasonably sized JavaScript, CSS, and images. The links and scripts and image tags for them are straight-forward in implementation. In the example below, we use the defaults to demonstrate how to implement the [Sakura css microframework]() instead of FastHTML's default use of Picocss.

```{.python filename="main.py" code-line-numbers="true"}
from fasthtml.fastapp import * 

app, rt = fast_app(
    default_hdrs=False,  # <8>
    hdrs=(
        Link(rel='stylesheet', href='assets/normalize.min.css', type='text/css'),
        Link(rel='stylesheet', href='assets/sakura.css', type='text/css'),  # <9>
        Style("p {color: red;}")  # <10>
))

@app.get("/")
def home():
    return Titled("FastHTML",
        P("Let's do this!"),
    )

run_uv()
```

8. By setting `default_hdrs` to `False`, FastHTML will not include picocss.
9. This will generate an HTML `<link>` tag for sourcing the css for Sakura.
10. If you want an inline styles, the `Style()` function will put the result into the HTML.

Check it out!

![](quickstart-sakura.png)

The HTML:

```python
<!doctype html>

<html>
  <head>
    <title>FastHTML</title>
    <link rel="stylesheet" href="assets/normalize.min.css" type="text/css">
    <link rel="stylesheet" href="assets/sakura.css" type="text/css">
    <style>p {color: red;}</style>    
  </head>
  <body>
<main class="container">
  <h1>FastHTML</h1>
  <p>Let&#x27;s do this!</p>
</main>
  </body>
</html>

```

## Advanced Static Media

If you want to customize your static media handler, you can define your own handler. 

```python
@rt("/{fname:path}.{ext:static}")
async def get(fname:str, ext:str): return FileResponse(f'public/{fname}.{ext}')
```

## Unwritten quickstart sections

1. Debug setting in fast_app
2. Rendering markdown
3. Code highlighting
4. Defining simple components of our own
5. Cookies
6. Sessions
7. Toast messages
8. BeforeWare and AfterWare

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()